In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Set

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.rename(columns= {'song': 'title'}, inplace=True)

In [4]:
df = df.sample(15000).drop('link', axis=1).reset_index(drop=True)


In [5]:
df.shape

(15000, 3)

In [16]:
df[df['artist'] == "Aerosmith"]

,artist,title,text
380,Aerosmith,Love Me Like A Bird Dog,savin ' all your money for a raini day limousi...
923,Aerosmith,Sweet Emotion,sweet emot sweet emot you talk about thing tha...
2402,Aerosmith,No More No More,blood stain the ivori of my daddi 's babi gran...
3037,Aerosmith,Hangman Jury,me and my old ladi sittin ' in the shade talki...
3127,Aerosmith,Joanie's Butterfly,"what a stormi night , when i met the poni , it..."
3210,Aerosmith,Spaced,space in time child of nine doin ' twenti year...
4117,Aerosmith,Kings And Queens,long ago in day untold were rule by lord of gr...
4605,Aerosmith,Line Up,if you think that you 're strong wan na fight ...
4821,Aerosmith,Kiss Your Past Good-Bye,finder keeper loser weep down on 42nd street b...
4826,Aerosmith,Never Loved A Girl,you 're a no good heartbreak you 're a liar an...


In [7]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'^\r\n', ' ', regex = True)

In [8]:
import nltk
# nltk.download('punkt_tab')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    # print(tokens)
    stemming = [stemmer.stem(w) for w in tokens]
    # print(stemming)
    return " ".join(stemming)

In [9]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [10]:
df.iloc[8005]['text']

"( first vers ) i thought love wa onli true in fairi tale , meant for someon els but not for me , love wa out to get me , ( baddabudumbadum ) that 's the way it seem , ( baddabudumbadum ) dissapoint haunt all my dream , [ choru : ] and then i saw her face , now i 'm a believ , not a trace , of doubt in ma ' mind , i 'm in love , oh ! i 'm a believ , i could n't leav her if i tri , [ vers 2 : ] i thought love wa more or less a given thing , seem the more you give , the less i got , what the use in tri , ( baddabudumbadum ) all you get is pain , ( baddabudumbadum ) when i need sunshin i got rain , [ choru : ]"

In [22]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Set

class MusicRecommendationEvaluator:
    def __init__(self, data: pd.DataFrame, k_values: List[int] = [5, 10], similarity_threshold: float = 0.3):
        self.data = data
        self.k_values = k_values
        self.similarity_threshold = similarity_threshold
        self.tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
        self.tfidf_matrix = None
        self.similarity_matrix = None
    
    def prepare_data(self):
        """Mempersiapkan TF-IDF matrix dan similarity matrix"""
        self.tfidf_matrix = self.tfidf.fit_transform(self.data['text'])
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)
        
    def get_recommendations(self, song_idx: int, k: int) -> List[int]:
        """Mendapatkan k rekomendasi teratas untuk sebuah lagu"""
        song_similarities = self.similarity_matrix[song_idx]
        similarity_scores = list(enumerate(song_similarities))
        sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        recommendations = [idx for idx, score in sorted_scores if idx != song_idx][:k]
        print(sorted_scores)
        return recommendations
    
    def create_ground_truth(self) -> Dict[int, Set[int]]:
        """
        Membuat ground truth berdasarkan similarity threshold
        Returns dictionary dengan key=index lagu dan value=set index lagu yang relevan
        """
        ground_truth = {}
        for i in range(len(self.data)):
            relevant_songs = set()
            similarities = self.similarity_matrix[i]
            for j, sim in enumerate(similarities):
                if i != j and sim >= self.similarity_threshold:
                    relevant_songs.add(j)
            ground_truth[i] = relevant_songs
        return ground_truth
    
    def calculate_precision_at_k(self, recommended: List[int], relevant: Set[int], k: int) -> float:
        """
        Menghitung Precision@K
        Precision = (Jumlah rekomendasi yang relevan) / (Jumlah rekomendasi yang diberikan)
        """
        if k == 0:
            return 0.0
        recommended_k = set(recommended[:k])
        relevant_and_recommended = len(recommended_k & relevant)
        return relevant_and_recommended / len(recommended_k)
    
    def calculate_recall_at_k(self, recommended: List[int], relevant: Set[int], k: int) -> float:
        """
        Menghitung Recall@K
        Recall = (Jumlah rekomendasi yang relevan) / (Total jumlah item yang relevan)
        """
        if len(relevant) == 0:
            return 0.0
        recommended_k = set(recommended[:k])
        relevant_and_recommended = len(recommended_k & relevant)
        return relevant_and_recommended / len(relevant)
    
    def calculate_average_precision(self, recommended: List[int], relevant: Set[int]) -> float:
        """Menghitung Average Precision"""
        if len(relevant) == 0:
            return 0.0
        
        running_sum = 0.0
        num_relevant = 0
        
        for i, song_idx in enumerate(recommended):
            if song_idx in relevant:
                num_relevant += 1
                precision_at_i = num_relevant / (i + 1)
                running_sum += precision_at_i
        
        return running_sum / len(relevant)
    
    def evaluate(self) -> Dict:
        """Mengevaluasi sistem rekomendasi dan menghitung semua metrics"""
        if self.similarity_matrix is None:
            self.prepare_data()
            
        ground_truth = self.create_ground_truth()
        
        # Print debugging information
        print("\nDebugging Information:")
        print(f"Similarity Threshold: {self.similarity_threshold}")
        print("\nGround Truth Summary:")
        for idx, relevant in list(ground_truth.items())[:5]:  # Print first 5 items
            print(f"Song {idx}: {len(relevant)} relevant items")
        
        results = {}
        for k in self.k_values:
            precisions = []
            recalls = []
            f1_scores = []
            aps = []
            
            print(f"\nEvaluating for k={k}")
            
            recommendations = self.get_recommendations(4826, k)
            relevant_songs = ground_truth[4826]
                
            if len(relevant_songs) == 0:
                continue
                
                
            # Calculate metrics
            precision = self.calculate_precision_at_k(recommendations, relevant_songs, k)
            recall = self.calculate_recall_at_k(recommendations, relevant_songs, k)
                
            # Calculate F1 Score
            if precision + recall > 0:
                f1 = 2 * (precision * recall) / (precision + recall)
            else:
                f1 = 0.0
                
            # Calculate AP
            ap = self.calculate_average_precision(recommendations, relevant_songs)
                
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            aps.append(ap)
            
            # Store results for this k
            results[f'k={k}'] = {
                'precision': np.mean(precisions),
                'recall': np.mean(recalls),
                'f1_score': np.mean(f1_scores),
                'map': np.mean(aps)
            }
        
        return results

    
    

In [23]:

evaluator = MusicRecommendationEvaluator(
    df, 
    k_values=[10, 15], 
    similarity_threshold=0.4  # Menggunakan threshold yang lebih tinggi
)
    
# Evaluasi dan tampilkan hasil
results = evaluator.evaluate()
    
print("\nHasil Evaluasi Sistem Rekomendasi Musik:")
print("----------------------------------------")
    
for k, metrics in results.items():
    print(f"\nMetrics untuk {k}:")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1-Score: {metrics['f1_score']:.4f}")
    print(f"MAP: {metrics['map']:.4f}")


Debugging Information:
Similarity Threshold: 0.4

Ground Truth Summary:
Song 0: 3 relevant items
Song 1: 6 relevant items
Song 2: 0 relevant items
Song 3: 0 relevant items
Song 4: 0 relevant items

Evaluating for k=3
[(4826, np.float64(1.0000000000000002)), (11502, np.float64(0.5147961865839206)), (2395, np.float64(0.4761849801366492)), (3595, np.float64(0.45734926034114154)), (11515, np.float64(0.4431169485402098)), (6128, np.float64(0.44143627271747155)), (14822, np.float64(0.4361001512850986)), (3250, np.float64(0.4305245158792047)), (11765, np.float64(0.4190390914046261)), (13863, np.float64(0.4170267312343198)), (14481, np.float64(0.41174233083567247)), (12019, np.float64(0.3928509109217351)), (8279, np.float64(0.39259695880016976)), (8611, np.float64(0.3925204632398207)), (4242, np.float64(0.3922995139204057)), (14678, np.float64(0.3824928626467279)), (14475, np.float64(0.37620883149037104)), (9579, np.float64(0.3736049587251673)), (13174, np.float64(0.37115766453924726)), (7374